# Advanced Topics: Analytics Tools

## Setup

In [ ]:
import os
import ibis

ibis.options.interactive = True

connection = ibis.sqlite.connect(os.path.join('data', 'geography.db'))

## Frequency tables

Ibis provides the `value_counts` API, just like pandas, for computing a frequency table for a table column or array expression. You might have seen it used already earlier in the tutorial. 

In [ ]:
countries = connection.table('countries')
countries.continent.value_counts()

This can be customized, of course:

In [ ]:
freq = (countries.group_by(countries.continent)
                 .aggregate([countries.count().name('# countries'),
                             countries.population.sum().name('total population')]))
freq

## Binning and histograms


Numeric array expressions (columns with numeric type and other array expressions) have `bucket` and `histogram` methods which produce different kinds of binning. These produce category values (the computed bins) that can be used in grouping and other analytics.

Some backends implement the `.summary()` method, which can be used to see the general distribution of a column.

Let's have a look at a few examples.

Alright then, now suppose we want to split the countries up into some buckets of our choosing for their population:

In [ ]:
buckets = [0, 1e6, 1e7, 1e8, 1e9]

The `bucket` function creates a bucketed category from the prices:

In [ ]:
bucketed = countries.population.bucket(buckets).name('bucket')

Let's have a look at the value counts:

In [ ]:
bucketed.value_counts()

The buckets we wrote down define 4 buckets numbered 0 through 3. The `NaN` is a pandas `NULL` value (since that's how pandas represents nulls in numeric arrays), so don't worry too much about that. Since the bucketing ends at 100000, we see there are 4122 values that are over 100000. These can be included in the bucketing with `include_over`:

In [ ]:
bucketed = (countries.population
            .bucket(buckets, include_over=True)
            .name('bucket'))
bucketed.value_counts()

The `bucketed` object here is a special **_category_** type

In [ ]:
bucketed.type()

Category values can either have a known or unknown **_cardinality_**. In this case, there's either 4 or 5 buckets based on how we used the `bucket` function.

Labels can be assigned to the buckets at any time using the `label` function:

In [ ]:
bucket_counts = bucketed.value_counts()

labeled_bucket = (bucket_counts.bucket
                  .label(['< 1M', '> 1M', '> 10M', '> 100M', '> 1B'])
                  .name('bucket_name'))

expr = (bucket_counts[labeled_bucket, bucket_counts]
        .sort_by('bucket'))
expr

Nice, huh?

Some backends implement `histogram(num_bins)`, a linear (fixed size bin) equivalent.